# Урок по Pandas на примере Titanic Dataset

В этом ноутбуке мы познакомимся с **основными функциями и приёмами работы** в библиотеке [Pandas](https://pandas.pydata.org/). 

В качестве примера будем использовать знаменитый датасет **Titanic**, содержащий информацию о пассажирах «Титаника» (пол, возраст, класс билета, порт отправления и т.д.), а также о том, выжили они или нет.

## Содержание
1. [Загрузка данных](#load_data)
2. [Описание столбцов](#columns_description)
3. [Базовое исследование DataFrame](#basic_exploration)
4. [Работа с пропущенными значениями](#missing_values)
5. [Фильтрация и группировка](#filter_groupby)
6. [Основные операции с DataFrame](#df_operations)
7. [Простая визуализация](#visualization)
8. [Дополнительные упражнения](#exercises)
9. [Заключение](#conclusion)

> **Примечание**: в данном ноутбуке мы рассматриваем только часть возможностей Pandas. Для более углублённого изучения обратите внимание на раздел «Дополнительно» в конце.

# 1. Загрузка данных <a name="load_data"></a>

Для начала импортируем Pandas, а также другие библиотеки, которые могут понадобиться (например, Matplotlib). Затем загрузим датасет Titanic.

Если у вас уже есть файл `titanic.csv` в той же папке, можно прочитать его командой `pd.read_csv('titanic.csv')`. 

Если вы используете Google Colab и хотите загрузить файл из своего компьютера:
1. Нажмите на иконку «папка» слева (раздел *Files*).
2. Нажмите кнопку «Upload to session storage» (или аналогичную) и выберите `titanic.csv`.
3. Используйте `pd.read_csv('titanic.csv')` для чтения.

Другие способы: подключиться к Google Drive или скачать CSV по ссылке, если он доступен.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Предположим, что файл 'titanic.csv' уже загружен в окружение.
# Если файл расположен в другой папке, укажите путь, например:
# df = pd.read_csv('/content/drive/MyDrive/titanic.csv')

df = pd.read_csv('titanic.csv')  # Измените путь, если требуется

# Узнаем форму (размер) DataFrame:
print("Форма DataFrame (строки, столбцы):", df.shape)

# Посмотрим на первые 5 строк, чтобы увидеть структуру:
df.head()

# 2. Описание столбцов <a name="columns_description"></a>

Ниже кратко описано, за что отвечает каждый столбец в датасете:

- **PassengerId**: Идентификатор пассажира
- **Survived**: Факт выживания (0 = нет, 1 = да)
- **Pclass**: Класс билета (1 = 1-й класс, 2 = 2-й класс, 3 = 3-й класс) — часто используется как признак социально-экономического статуса
- **Name**: Имя пассажира
- **Sex**: Пол (male, female)
- **Age**: Возраст (в годах)
- **SibSp**: Количество братьев/сестёр или супругов на борту
- **Parch**: Количество родителей или детей на борту
- **Ticket**: Номер билета
- **Fare**: Стоимость билета
- **Cabin**: Номер каюты
- **Embarked**: Порт посадки (C = Cherbourg, Q = Queenstown, S = Southampton)

С этими признаками мы и будем работать для иллюстрации возможностей Pandas.

# 3. Базовое исследование DataFrame <a name="basic_exploration"></a>

После загрузки данных важно понять, какие у нас есть столбцы, какие типы данных, есть ли пропущенные значения и т.д. Для этого в Pandas есть несколько удобных методов.

In [ ]:
# Общая информация об объёмах и типах данных:
df.info()

`info()` показывает, сколько всего строк (Non-Null Count), где есть пропуски, а также типы столбцов.

Другой важный метод — `describe()`, который даёт базовую статистику по числовым данным.

In [ ]:
df.describe()

Можно также посмотреть на названия столбцов и, например, на количество уникальных значений.

In [ ]:
print("Столбцы:", df.columns.tolist())
print("\nКоличество уникальных значений в каждом столбце:")
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"{col}: {unique_count}")

# 4. Работа с пропущенными значениями <a name="missing_values"></a>

В реальных данных часто встречаются пропущенные значения (NaN). Посмотрим, сколько таких значений в каждом столбце:

In [ ]:
df.isnull().sum()

Видим, что, например, в столбце **Age** и **Cabin** есть пропуски. 

Способы работы с пропусками:
- Удалять строки с `NaN` (но тогда теряем информацию)
- Заполнять средним, медианой, либо специальным значением
- Пытаться предсказывать пропуски

Здесь для примера заполним пропуски по возрасту средним возрастом (не лучший вариант с точки зрения реального анализа, но в качестве иллюстрации).

In [ ]:
mean_age = df['Age'].mean()
df['Age'].fillna(mean_age, inplace=True)

print("После заполнения пропусков в Age:")
df.isnull().sum()

Так мы убрали пропуски в столбце **Age**, но пропуски в **Cabin** остались — их тоже можно обработать, если нужно.


# 5. Фильтрация и группировка <a name="filter_groupby"></a>

Одно из ключевых достоинств Pandas — гибкая фильтрация и группировка данных.

## 5.1 Фильтрация (Filtering)

Допустим, хотим выбрать только пассажиров, которым меньше 18 лет (дети).

In [ ]:
children_df = df[df['Age'] < 18]
children_df.head()

А теперь выберем только женщин 1-го класса:

In [ ]:
women_first_class = df[(df['Sex'] == 'female') & (df['Pclass'] == 1)]
women_first_class.head()

## 5.2 Группировка (GroupBy)

С помощью `groupby()` можно быстро агрегировать данные. Например, посмотрим на средний возраст пассажиров в каждом классе.

In [ ]:
df.groupby('Pclass')['Age'].mean()

А теперь посмотрим на выживаемость (Survived) по полу пассажиров:

In [ ]:
df.groupby('Sex')['Survived'].mean()

То есть доля выживших женщин и мужчин в нашем датасете.

# 6. Основные операции с DataFrame <a name="df_operations"></a>

Рассмотрим несколько часто используемых операций:
- Добавление новых столбцов
- Удаление столбцов или строк
- Сортировка
- Слияние и соединение (merge, join) — в этом ноутбуке лишь кратко упомянем

## 6.1 Добавление новых столбцов

Например, посчитаем, сколько всего родственников (SibSp + Parch) было у каждого пассажира:

In [ ]:
df['Relatives'] = df['SibSp'] + df['Parch']
df[['SibSp','Parch','Relatives']].head(10)

## 6.2 Удаление столбцов или строк

Иногда нужно удалить лишнюю информацию. Например, удалим столбец `Ticket`, предположим, что он нам не нужен:

In [ ]:
df.drop(columns='Ticket', inplace=True)
df.head(3)

## 6.3 Сортировка

Можно отсортировать пассажиров по возрасту (по возрастанию):

In [ ]:
df_sorted = df.sort_values(by='Age', ascending=True)
df_sorted.head(10)

# 7. Простая визуализация <a name="visualization"></a>

Pandas позволяет быстро строить простые графики на основе данных. Для более сложной визуализации часто используют **Matplotlib** и **Seaborn**.

## 7.1 Гистограмма возраста

Посмотрим распределение возраста пассажиров на гистограмме.

In [ ]:
df['Age'].plot(kind='hist', bins=20, edgecolor='black', title='Распределение возраста')
plt.xlabel('Возраст')
plt.show()

## 7.2 Диаграмма баров (bar plot) выживаемости по полу

Построим столбиковую диаграмму, показывающую долю выживших мужчин и женщин.

In [ ]:
survival_by_sex = df.groupby('Sex')['Survived'].mean()
survival_by_sex.plot(kind='bar', color=['orange','blue'], title='Доля выживших по полу')
plt.ylabel('Доля выживших')
plt.show()

# 8. Дополнительные упражнения <a name="exercises"></a>

1. **Проанализируйте выживаемость по классам** (Pclass). Постройте столбиковую диаграмму.
2. **Выведите** 10 самых старших пассажиров, которые выжили.
3. **Посчитайте** среднюю стоимость билета (Fare) для каждого порта посадки (Embarked).
4. **Создайте** новый столбец, который отражает, был ли пассажир в возрасте от 0 до 12 ("Child"), от 13 до 19 ("Teen") или старше 19 ("Adult").
5. **Постройте** гистограмму распределения столбца `Relatives` (который мы добавили).

# 9. Заключение <a name="conclusion"></a>

В этом ноутбуке мы рассмотрели основные методы работы с Pandas:
- Загрузка данных (`pd.read_csv`)
- Получение базовой информации (`.info()`, `.describe()`, `.head()`, и т.д.)
- Фильтрация и группировка (`.groupby()`, условия)
- Работа с пропущенными значениями (`isnull()`, `fillna()`)
- Создание и удаление столбцов (`df['NewCol'] = ...`, `drop()`)
- Сортировка данных (`sort_values()`)
- Простейшая визуализация (гистограмма, bar plot)

## Куда двигаться дальше?
- Изучить **merge**, **join**, **concat** для объединения DataFrame.
- Освоить библиотеку **Matplotlib** глубже (или **Seaborn**, **Plotly**) для более продвинутой визуализации.
- Практиковаться на разных датасетах (Iris, Wine, Housing).
- Изучить методы очистки и подготовки данных (в том числе *feature engineering*).

### Полезные ссылки
- [Pandas Documentation](https://pandas.pydata.org/docs/)
- [Kaggle Datasets](https://www.kaggle.com/datasets) — множество публичных датасетов
- [Matplotlib Documentation](https://matplotlib.org/stable/contents.html)
- [Seaborn Documentation](https://seaborn.pydata.org/)

Удачи в анализе данных!